# 12章 分類の手法を学ぼう

In [ ]:
import pandas as pd


In [ ]:
from sklearn.datasets import load_breast_cancer


In [ ]:
dataset = load_breast_cancer()
display(dataset)


In [ ]:
print(dataset.DESCR)


In [ ]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df.head()

In [ ]:
df['class'] = dataset.target
display(df.head())
display(df.shape)


In [ ]:
display(df.info())

In [ ]:
df.hist(figsize=(20, 15), bins=30)


## 5. 学習データとテストデータへの分割

In [ ]:
X = df.drop(columns=['class']).to_numpy()
y = df['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## 6. 予測モデルの学習

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)

In [ ]:
model.fit(X_train, y_train)

## 7. 予測モデルの評価

In [ ]:
y_pred = model.predict(X_test)
display(y_pred)
display(y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

## 8. 予測

## 9. 設定したゴールに対する考察

In [ ]:
from sklearn.tree import export_text
# 要素名の取得
names = dataset.feature_names
# 要素名をリスト形式に変換
names_list = names.tolist()
# 条件分岐構造を出力
print(export_text(model, decimals=3, feature_names=names_list))


## 12.4 予測モデルの改善

In [ ]:
model = DecisionTreeClassifier(
    max_depth=2,
    max_leaf_nodes=3,
    min_samples_leaf=10,
    random_state=0)

In [ ]:
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)
display(y_pred)
print(classification_report(y_test, y_pred))


In [ ]:
print(export_text(model, decimals=3, feature_names=names_list))


# 実践1 (pandasの復習)
* 「7. 予測モデルの評価」で実行したclassification_reportをdataframeに格納してください。その後csvとして保存してください。
* 実務でもdataframeに格納してcsvへ保存するケースはあります。

# 実践2(matplotlib, seabornの復習 & 一部発展)
* load_breast_cancerを講義テキストでは
```
df.hist(figsize=(20, 15), bins=30)
```
として実施しました。
* これをseabornのstripplotを使って同様に表示してください。その際、全特徴量を1つのfigとして表示し、図を保存してください。
* ポイント: subplotsを用いること & for文を使ってstripplotをおこなう。
* 参考) 実務でもこのように全特徴量に対して一斉の可視化を行うケース(pandasの可視化だけでは対応できないケース)はあります。一度ロジックを組み立てておけば汎用的に使えるので覚えておくと便利です。
* 更に発展させると、この処理のベースを関数/クラスのメソッド化しておき、dataframeを渡すだけにしておくと更に利便性が高まります。


# 実践3
- 「7. 予測モデルの評価」の流れで混同行列を作成し表示してください。(classification_reportの元となる行列で実務でも利用します)

# 実践4(マルチクラス分類)
* irisデータを取得してください。
```
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target
```
* irisデータに対して学習・推論を実施してください。モデルは任意で構いませんが、木構造以外のアルゴリズムの場合は標準化を視野に入れてください(厳密にはデータによっては標準化が不要な場合もありますが、基本は実施すべきです)。木構造の場合はexport_text等で条件分岐を出力してください。
* 混同行列や評価指標を算出してください。


講義の補足に関する参考)
average='macro'は、多クラス分類問題において、各クラスの指標の単純平均を取ることを意味しています。
具体的には、以下のように計算されます。

- recallの場合

    - クラス1のrecallを計算
    - クラス2のrecallを計算
    - クラス3のrecallを計算
    - 上記3つのrecallの平均を取る


- precisionの場合も同様に

    - クラス1のprecisionを計算
    - クラス2のprecisionを計算
    - クラス3のprecisionを計算

上記3つのprecisionの平均を取る



- 一方で、average='micro'とすると、真の正例数と予測された正例数の比から計算されます。classification_reportsのweighted_avgとはまた別です。
- クラスが非常に偏っているデータセットの場合、microとmacroで値が大きく異なる可能性があります。
- どちらを選ぶかは状況次第ですが、通常は以下のようなケースで使い分けられています。

- average='macro' : 各クラスが同等に重要で、偏りのない評価が必要な場合
- average='micro' : 総合的な性能評価が重要で、クラスの偏りが問題ない場合

今回はマルチクラス分類なので、各クラスが同等に重要と考え、average='macro'を選びました。

データの特性によっては、microの方が適切な場合もあるかもしれません。


それぞれの平均値の計算方法をirisデータセットの実際の値を使って説明します。

irisデータセットにはsetosa、versicolor、virginicaの3クラスがあり、classification_reportから得られる値は以下の通りです。
```
Copy code              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.94      1.00      0.97        16
   virginica       1.00      0.91      0.95        11
    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.97      0.97      0.97        40

```
__macro avg__


precision = (1.00 + 0.94 + 1.00) / 3 = 0.98

recall = (1.00 + 1.00 + 0.91) / 3 = 0.97

f1-score = (1.00 + 0.97 + 0.95) / 3 = 0.97

macro avgは簡単に各クラスのスコアの平均を取ります。

__weighted avg__
```
precision = (1.00 * 13 + 0.94 * 16 + 1.00 * 11) / 40 = 0.97
recall = (1.00 * 13 + 1.00 * 16 + 0.91 * 11) / 40 = 0.97
f1-score = (1.00 * 13 + 0.97 * 16 + 0.95 * 11) / 40 = 0.97
```

weighted avgはサンプル数(support列)を加味して、出現頻度に応じた重み付け平均を計算します。

__micro avg__
```
precision = TP / (TP + FP)
= (13 + 16 + 10) / (13 + 16 + 10 + 0 + 0 + 1)
= 39 / 40
= 0.975
recall = TP / (TP + FN)
= (13 + 16 + 10) / (13 + 16 + 11)
= 39 / 40
= 0.975
f1-score = 2 * precision * recall / (precision + recall)
= 2 * 0.975 * 0.975 / (0.975 + 0.975)
= 0.975
```
micro avgは全体の真陽性(TP)、偽陽性(FP)、偽陰性(FN)から算出されます。クラス間の出現頻度は考慮されません。

このように、macro avgはクラス間の偏りを無視、weighted avgはサンプル数の偏りを考慮、micro avgは全体の集計値から算出される、という違いがあります。

状況に応じて適切な指標を選ぶ必要があります。
